In [2]:
!pip install requests

In [3]:
import requests
import json
import os

In [4]:
from getpass import getpass
import requests
import os

hard_reset = False  ## <-- Set to True if you want to reset your NVIDIA_API_KEY
while "nvapi-" not in os.environ.get("NVIDIA_API_KEY", "") or hard_reset:
    ## Try to set NVIDIA_API_KEY as part of docker_router routine.
    ##  When running in course container, this helps to save your API key between sessions.
    try:
        assert not hard_reset
        response = requests.get("http://docker_router:8070/get_key").json()
        assert response.get('nvapi_key')
    except: response = {'nvapi_key' : getpass("NVIDIA API Key: ")}
    os.environ["NVIDIA_API_KEY"] = response.get("nvapi_key")
    try: requests.post("http://docker_router:8070/set_key/", json={'nvapi_key' : os.environ["NVIDIA_API_KEY"]}).json()
    except: pass
    hard_reset = False
    if "nvapi-" not in os.environ.get("NVIDIA_API_KEY", ""):
        print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")

print(f"Retrieved NVIDIA_API_KEY beginning with \"{os.environ.get('NVIDIA_API_KEY')[:9]}...\"")

NVIDIA API Key: ··········
Retrieved NVIDIA_API_KEY beginning with "nvapi-F82..."


In [5]:

headers = {
    "Authorization": f"Bearer {os.environ.get('NVIDIA_API_KEY')}",
    "accept": "text/event-stream",
    "content-type": "application/json",
}


In [6]:
def get_stream_token(entry: bytes):
    """Utility: Coerces out ['choices'][0]['delta'][content] from the bytestream"""
    if not entry: return ""
    entry = entry.decode('utf-8')
    if entry.startswith('data: '):
        try: entry = json.loads(entry[5:])
        except ValueError: return ""
    return entry.get('choices', [{}])[0].get('delta', {}).get('content')


In [7]:
invoke_url = "https://api.nvcf.nvidia.com/v2/nvcf/pexec/functions/404"

In [8]:
payload = {
    "prompt": "This is the input text or prompt for the language model.",
    "max_tokens": 100,
    "temperature": 0.7,
    "stop": ["###"]
}

In [9]:
response = requests.post(invoke_url, headers=headers, json=payload, stream=True)

In [10]:
response.raise_for_status()

HTTPError: 400 Client Error: Bad Request for url: https://api.nvcf.nvidia.com/v2/nvcf/pexec/functions/404

In [11]:
for line in response.iter_lines():
    print(get_stream_token(line), end="")
    if line: print(line.decode("utf-8"))

AttributeError: 'str' object has no attribute 'get'

#ChatNVIDIA

In [12]:
# %pip install pillow==10.0.1
%pip install --upgrade imageio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.5/313.5 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: imageio
    Found existing installation: imageio 2.31.6
    Uninstalling imageio-2.31.6:
      Successfully uninstalled imageio-2.31.6


In [13]:
%pip install -q langchain-nvidia-ai-endpoints


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 16.7 MB/s eta 0:00:00


In [14]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_nvidia_ai_endpoints._common import NVEModel  ## Backend Model

## Using the backbone NVIDIA Endpoints client, which makes the calls as you saw above
NVEModel().available_models

{'ai-sdxl-turbo': 'f886140c-424e-4c82-a841-99e23f9ae35d',
 'ai-microsoft-kosmos-2': '6018fed7-f227-48dc-99bc-3fd4264d5037',
 'ai-mixtral-8x7b-instruct': 'a1e53ece-bff4-44d1-8b13-c009e5bf47f6',
 'ai-gemma-7b': 'a13e3bed-ca42-48f8-b3f1-fbc47b9675f9',
 'ai-codegemma-7b': '7dfc10a8-3cc4-448e-97c1-2213308dc222',
 'playground_neva_22b': '8bf70738-59b9-4e5f-bc87-7ab4203be7a0',
 'ai-molmim-generate': '72be0b68-179f-412c-ac03-9a481f78cb9f',
 'ai-diffdock': 'f3dda972-561a-4772-8c09-873594b6fb72',
 'ai-codellama-70b': 'f6b06895-d073-4714-8bb2-26c09e9f6597',
 'playground_clip': '8c21289c-0b18-446d-8838-011b7249c513',
 'ai-arctic-embed-l': '1528a0ad-205a-46ac-a783-94e2372586a9',
 'ai-llama3-8b': 'a5a3ad64-ec2c-4bfc-8ef7-5636f26630fe',
 'ai-esmfold': 'a68c59e0-47a6-4a50-bf64-6d88766d56bf',
 'ai-stable-diffusion-xl': 'c1b63bb0-448b-4e53-b2a7-fb0b3723cbe2',
 'playground_sdxl': '89848fb8-549f-41bb-88cb-95d6597044a4',
 'playground_mamba_chat': '381be320-4721-4664-bd75-58f8783b43c7',
 'playground_llama2_

In [15]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

## NOTE: "playground_" prefix is optional for our client
chat = ChatNVIDIA(model='llama2_70b')
chat.invoke("Hello! How's it going?")

ChatMessage(content="Hello! I'm doing well, thanks for asking. How about you? Is there anything you'd like to chat about or ask? I'm here to help with any questions you might have.", response_metadata={'role': 'assistant', 'content': "Hello! I'm doing well, thanks for asking. How about you? Is there anything you'd like to chat about or ask? I'm here to help with any questions you might have.", 'token_usage': {'completion_tokens': 42, 'prompt_tokens': 16, 'total_tokens': 58}, 'model_name': 'llama2_70b'}, id='run-fb014708-b91c-460e-a8e8-46916f63291b-0', role='assistant')

##Environment Setup


In [16]:
%pip install -q langchain langchain-nvidia-ai-endpoints gradio typing_extensions>=4.8.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.


In [17]:
from langchain_nvidia_ai_endpoints._common import NVEModel

In [18]:
from getpass import getpass
import requests
import os

hard_reset = False  ## <-- Set to True if you want to reset your NVIDIA_API_KEY
while "nvapi-" not in os.environ.get("NVIDIA_API_KEY", "") or hard_reset:
    try:
        assert not hard_reset
        response = requests.get("http://docker_router:8070/get_key").json()
        assert response.get('nvapi_key')
    except: response = {'nvapi_key' : getpass("NVIDIA API Key: ")}
    os.environ["NVIDIA_API_KEY"] = response.get("nvapi_key")
    try: requests.post("http://docker_router:8070/set_key/", json={'nvapi_key' : os.environ["NVIDIA_API_KEY"]}).json()
    except: pass
    hard_reset = False
    if "nvapi-" not in os.environ.get("NVIDIA_API_KEY", ""):
        print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")

print(f"Retrieved NVIDIA_API_KEY beginning with \"{os.environ.get('NVIDIA_API_KEY')[:9]}...\"")
from langchain_nvidia_ai_endpoints._common import NVEModel
NVEModel().available_models

Retrieved NVIDIA_API_KEY beginning with "nvapi-F82..."


{'ai-parakeet-ctc-riva': '22164014-a6cc-4a6f-b048-f3a303e745bb',
 'ai-llama2-70b': '2fddadfb-7e76-4c8a-9b82-f7d3fab94471',
 'playground_gemma_7b': '1361fa56-61d7-4a12-af32-69a3825746fa',
 'playground_llama2_code_70b': '2ae529dc-f728-4a46-9b8d-2697213666d8',
 'playground_mistral_7b': '35ec3354-2681-4d0e-a8dd-80325dcf7c63',
 'playground_yi_34b': '347fa3f3-d675-432c-b844-669ef8ee53df',
 'playground_nemotron_qa_8b': '0c60f14d-46cb-465e-b994-227e1c3d5047',
 'ai-sea-lion-7b-instruct': '02f84bf4-c1a1-489b-a9de-ac3e8dcdec14',
 'playground_gemma_2b': '5bde8f6f-7e83-4413-a0f2-7b97be33988e',
 'ai-molmim-generate': '72be0b68-179f-412c-ac03-9a481f78cb9f',
 'ai-mixtral-8x22b-instruct': '710c92d0-7c98-46d6-b5ae-07e84bcaa5d3',
 'ai-mistral-large': '767b5b9a-3f9d-4c1d-86e8-fa861988cee7',
 'ai-microsoft-kosmos-2': '6018fed7-f227-48dc-99bc-3fd4264d5037',
 'playground_seamless': '72ad9555-2e3d-4e73-9050-a37129064743',
 'ai-arctic': '7408b6b5-09e7-4ae5-a3fe-2db063e4e609',
 'playground_nv_llama2_rlhf_70b': 

**Runnable** - which is simply an object that wraps a function. Allow dictionaries to be implicitly converted to Runnables and let a pipe | operator create a Runnable that passes data from the left to the right (i.e. fn1 | fn2 is a Runnable), and you have a simple way to specify complex logic!



In [19]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from functools import partial


In [20]:
identity = RunnableLambda(lambda x: x)  ## Or RunnablePassthrough works

Given an arbitrary function, you can make a runnable with it


In [21]:
def print_and_return(x, preface=""):
    print(f"{preface}{x}")
    return x

rprint0 = RunnableLambda(print_and_return)


In [22]:
rprint1 = RunnableLambda(partial(print_and_return, preface="1): "))


And you can use the same idea to make your own custom Runnable generator


In [23]:
def RPrint(preface=""):
    return RunnableLambda(partial(print_and_return, preface=preface))

In [24]:
chain1 = identity | rprint0
chain1.invoke("Hello World!")
print()

Hello World!



 Chaining that one in as well


In [25]:
output = (
    chain1
    | rprint1
    | RPrint("2: ")
).invoke("Welcome Home!")

print("\nOutput:", output)

Welcome Home!
1): Welcome Home!
2: Welcome Home!

Output: Welcome Home!


##Dictionary Pipelines with Chat Models

In [26]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

Non streaming interface

In [27]:
## Simple Chat Pipeline
chat_llm = ChatNVIDIA(model="llama2_70b")

prompt = ChatPromptTemplate.from_messages([
    ("system", "Only respond in rhymes"),
    ("user", "{input}")
])

rhyme_chain = prompt | chat_llm | StrOutputParser()
user_input = input("You need a rhym about : ")
print(rhyme_chain.invoke({"input" : user_input}))

You need a rhym about : Mom
Hi, Mom, how's your day?
I'm feeling bright, in a playful way.
The sun is shining bright,
A beautiful sight.
I hope you're having a great day,
In every possible way.


## gradio for a better chat interface

In [28]:
!pip install gradio

In [29]:
import gradio as gr

 Non-streaming Interface like that shown above


In [30]:
# def rhyme_chat(message, history):
#     return rhyme_chain.invoke({"input" : message})

# gr.ChatInterface(rhyme_chat).launch()


Streaming Interface

In [31]:
def rhyme_chat_stream(message, history):
    buffer = ""
    for token in rhyme_chain.stream({"input": message}):
        buffer += token
        yield buffer

In [32]:
iface = gr.Interface(
    fn=rhyme_chat_stream,
    inputs="text",
    outputs="text",
    live=True,
    description="Streaming rhyming chat interface"
)

/usr/local/lib/python3.10/dist-packages/gradio/utils.py:985: UserWarning: Expected 2 arguments for function <function rhyme_chat_stream at 0x7e084435a440>, received 1.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:989: UserWarning: Expected at least 2 arguments for function <function rhyme_chat_stream at 0x7e084435a440>, received 1.
  warnings.warn(


In [33]:
demo.launch(share=False, debug=True)


NameError: name 'demo' is not defined

In [34]:
instruct_llm = ChatNVIDIA(model="llama2_70b")

Zero-shot classification prompt and chain


In [35]:
zsc_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "Pick the most likely topic of the sentence. Choose an one option of the following: {options}. Only one word-answers"
    )),
    ("user", "[Options : {options}] {input} = ")
])

zsc_chain = zsc_prompt | instruct_llm | StrOutputParser()


In [36]:
def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options})


In [37]:
print("-" * 80)
print(zsc_call("Should I take the next exit, or keep going to the next one?"))

print("-" * 80)
print(zsc_call("I get seasick, so I think I'll pass on the trip"))

print("-" * 80)
print(zsc_call("I'm scared of heights, so flying probably isn't for me"))

print("-"*80)
print(zsc_call("I have to travel for 120km and i have less amount of money with me. Which mode of transport should be benifitial for me"))

--------------------------------------------------------------------------------
'car'
--------------------------------------------------------------------------------
boat
--------------------------------------------------------------------------------
airplane
--------------------------------------------------------------------------------
Bike


##Multi-Component Chains

The previous example showed how we can coerce a dictionary into a string by passing it through a prompt->LLM chain, so that's one easy structure to motivate the container choice. But is it just as easy to convert the string output back up to a dictionary?

Yes, it is! The simplest way is actually to use the LCEL "implicit runnable" syntax, which allows you to use a dictionary of functions (including chains) as a runnable that runs each function and maps the value to the key in the output dictionary.

In [38]:
def make_dictionary(v, key):
    if isinstance(v, dict):
        return v
    return {key : v}

def RInput(key='input'):
    '''Coercing method to mold a value (i.e. string) to in-like dict'''
    return RunnableLambda(partial(make_dictionary, key=key))


def ROutput(key='output'):
    '''Coercing method to mold a value (i.e. string) to out-like dict'''
    return RunnableLambda(partial(make_dictionary, key=key))

In [39]:
from operator import itemgetter

In [40]:
up_and_down = (
    RPrint("A: ")
    ## Custom ensure-dictionary process
    | RInput()
    | RPrint("B: ")
    ## Pull-values-from-dictionary utility
    | itemgetter("input")
    | RPrint("C: ")
    ## Anything-in Dictionary-out implicit map
    | {
        'word1' : (lambda x : x.split()[0]),
        'word2' : (lambda x : x.split()[1]),
        'words' : (lambda x: x),  ## <- == to RunnablePassthrough()
    }
    | RPrint("D: ")
    | itemgetter("word1")
    | RPrint("E: ")
    ## Anything-in anything-out lambda application
    | RunnableLambda(lambda x: x.upper())
    | RPrint("F: ")
    ## Custom ensure-dictionary process
    | ROutput()
)


In [41]:
up_and_down.invoke({"input" : "Hello World"})

A: {'input': 'Hello World'}
B: {'input': 'Hello World'}
C: Hello World
D: {'word1': 'Hello', 'word2': 'World', 'words': 'Hello World'}
E: Hello
F: HELLO


{'output': 'HELLO'}

the dictionary enforcement methods make it easy to make the following syntax equivalent


In [42]:
up_and_down.invoke("Hello World")

A: Hello World
B: {'input': 'Hello World'}
C: Hello World
D: {'word1': 'Hello', 'word2': 'World', 'words': 'Hello World'}
E: Hello
F: HELLO


{'output': 'HELLO'}

 Implement the rest of the rhyme_chat2_stream method such that the agent is able to function normally.

In [43]:
from copy import deepcopy

In [44]:
prompt1 = ChatPromptTemplate.from_messages([
    ("system", "Only respond in rhymes"),
    ("user", "{input}")
])
prompt2 =  ChatPromptTemplate.from_messages([
    ("system", (
        "Only responding in rhyme, change the topic of the input poem to be about {topic}!"
        " Make it happy! Try to keep the same sentence structure, but make sure it's easy to recite!"
        " Try not to rhyme a word with itself."
    )),
    ("user", "{input}")
])

In [45]:
chain1 = prompt1 | instruct_llm | StrOutputParser()  ## only expects input
chain2 = prompt2 | instruct_llm | StrOutputParser()  ## expects both input and topic

In [46]:
def rhyme_chat2_stream(message, history, return_buffer=True):
    '''This is a generator function, where each call will yield the next entry'''

    first_poem = None
    for entry in history:
        if entry[0] and entry[1]:
            ## If a generation occurred as a direct result of a user input,
            ##  keep that response (the first poem generated) and break out
            first_poem = entry[1]
            break

    if first_poem is None:
        ## First Case: There is no initial poem generated. Better make one up!

        buffer = "Oh! I can make a wonderful poem about that! Let me think!\n\n"
        yield buffer

        ## iterate over stream generator for first generation
        inst_out = ""
        chat_gen = chain1.stream({"input" : message})
        for token in chat_gen:
            inst_out += token
            buffer += token
            yield buffer if return_buffer else token

        passage = "\n\nNow let me rewrite it with a different focus! What should the new focus be?"
        buffer += passage
        yield buffer if return_buffer else passage

    else:
        ## Subsequent Cases: There is a poem to start with. Generate a similar one with a new topic!

        buffer = f"Sure! Here you go!\n\n"
        yield buffer

        topic = message
        buffer += f"Changing the topic to: {topic}\n\n"
        yield buffer

        inst_out = ""
        chat_gen = chain2.stream({"input": first_poem, "topic": topic})
        ## iterate over stream generator for second generation
        for token in chat_gen:
            inst_out += token
            buffer += token
            yield buffer if return_buffer else token

        passage = "\n\nThis is fun! Give me another topic!"
        buffer += passage
        yield buffer if return_buffer else passage

###Below: This is a small-scale simulation of the gradio routine.

In [47]:
def queue_fake_streaming_gradio(chat_stream, history = [], max_questions=5):

    ## Mimic of the gradio initialization routine, where a set of starter messages can be printed off
    for human_msg, agent_msg in history:
        if human_msg: print("\n[ Human ]:", human_msg)
        if agent_msg: print("\n[ Agent ]:", agent_msg)

    ## Mimic of the gradio loop with an initial message from the agent.
    for _ in range(max_questions):
        message = input("\n[ Human ]: ")
        print("\n[ Agent ]: ")
        history_entry = [message, ""]
        for token in chat_stream(message, history, return_buffer=False):
            print(token, end='')
            history_entry[1] += token
        history += [history_entry]
        print("\n")


In [49]:
history = [[None, "Let me help you make a poem! What would you like for me to write?"]]

## Simulating the queueing of a streaming gradio interface, using python input
queue_fake_streaming_gradio(
    chat_stream = rhyme_chat2_stream,
    history = history
)


[ Agent ]: Let me help you make a poem! What would you like for me to write?

[ Human ]: Mom

[ Agent ]: 
Oh! I can make a wonderful poem about that! Let me think!

Hi, Mom, how's your day?
I'm feeling bright, in a playful way.
The sun is shining bright and bold,
A beautiful day for young and old.

Now let me rewrite it with a different focus! What should the new focus be?


[ Human ]: GirlFriend

[ Agent ]: 
Sure! Here you go!

Sure! Here you go!

Changing the topic to: GirlFriend

Sure, I can help you with that! Here's a revised version of the poem, focusing on the topic of a girlfriend, with a happy and playful tone:

Hey, Girlfriend, how's your day?
I'm feeling bright, in a playful way.
You're the sunshine in my sky,
A beautiful sight, oh my!

I hope you like it! Let me know if you have any other requests or questions.

This is fun! Give me another topic!


[ Human ]: Breakup

[ Agent ]: 
Sure! Here you go!

Sure! Here you go!

Changing the topic to: Breakup

Sure, I can help you 

## FastAPI

In [ ]:
%pip install pydantic==1.10.13

In [ ]:
!pip install langserve


In [ ]:
!pip install sse_starlette

In [ ]:
!pip install nest_asyncio

In [ ]:
%%writefile server_app.py

In [ ]:
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langserve import add_routes

In [ ]:
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple api server using Langchain's Runnable interfaces",
)

add_routes(
    app,
    instruct_llm,
    path="/basic_chat",
)

import nest_asyncio
nest_asyncio.apply()

# Run the app using uvicorn
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=9012)

In [ ]:
!python langserve_client.py

# Running State Chains

##Keeping Variables Flowing

In the previous examples, we were able to implement interesting logic in our standalone chains by creating, mutating, and consuming states. These states were passed around as dictionaries with descriptive keys and useful values, and the values would be used to supply follow-up routines with the info they need to operate!

Recall the zero-shot classification

In [ ]:
%time

 Zero-shot classification prompt and chain


In [50]:
zsc_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "One word answer. Output the most relevant topic option. [Options : {options}]"
    )),
    ("user", "{input} = ")
])

In [51]:
def EnumParser(*idxs):
    '''Method that pulls out values from a model that outputs numbered entries'''
    idxs = idxs or [slice(0, None, 1)]
    entry_parser = lambda v: v if (' ' not in v) else v[v.index(' '):]
    out_lambda = lambda x: [entry_parser(v).strip() for v in x.split("\n")]
    return StrOutputParser() | RunnableLambda(lambda x: itemgetter(*idxs)(out_lambda(x)))

In [52]:
instruct_llm = ChatNVIDIA(model="llama2_70b") | EnumParser(0)

In [53]:
zsc_chain = zsc_prompt | instruct_llm

In [54]:
def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options})

In [55]:

print("-" * 80)
print(zsc_call("Should I take the next exit, or keep going to the next one?"))

print("-" * 80)
print(zsc_call("I get seasick, so I think I'll pass on the trip"))

print("-" * 80)
print(zsc_call("I'm scared of heights, so flying probably isn't for me"))

print("-"*80)
print(zsc_call("I have to travel for 120km and i have less amount of money with me. Which mode of transport should be benifitial for me"))

--------------------------------------------------------------------------------
Bike
--------------------------------------------------------------------------------
boat
--------------------------------------------------------------------------------
'bike'
--------------------------------------------------------------------------------
Bike


In [56]:
gen_prompt = ChatPromptTemplate.from_template(
    "Make a new sentence about the the following topic: {topic}. Be creative!"
)

gen_chain = gen_prompt | instruct_llm

input_msg = "I get seasick, so I think I'll pass on the trip"
options = ["car", "boat", "airplane", "bike"]

({'topic' : zsc_chain} | gen_chain).invoke({"input" : input_msg, "options" : options})

'sleek sailboat glided across the turquoise waters, its white sails billowing in the gentle breeze as it carried a group of laughing friends on a sun-kissed adventure to a secluded island paradise.'

However, it's a bit problematic when you want to keep the information flowing since we lose the topic and input variables in generating our response. If we wanted do something with both the output and the input, we'd need a to make sure that both variables pass through.

Lucky for us, we can use the mapping runnable (i.e. interpretted from a dictionary or using manual RunnableMap) to pass both of the variables through by assigning the output of our chain to just a single key and letting the other keys propagate as desired. Alternatively, we could also use RunnableAssign to merge the state-consuming chain's output with the input dictionary by default.

With this technique, we can propagate whatever we want through our chain system:

In [57]:
from langchain.schema.runnable import RunnableBranch, RunnablePassthrough
from langchain.schema.runnable.passthrough import RunnableAssign
from functools import partial

In [58]:
def RPrint(preface=""):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))


In [59]:
big_chain = (
    RPrint("State: ")
    ## Manual mapping. Can be useful sometimes and inside branch chains
    | {'input' : lambda d: d.get('input'), 'topic' : zsc_chain}
    | RPrint("State: ")
    ## RunnableAssign passing. Better for running state chains by default
    | RunnableAssign({'generation' : gen_chain})
    | RPrint("State: ")
    ## Using the input and generation together
    | RunnableAssign({'combination' : (
        ChatPromptTemplate.from_template(
            "Consider the following passages:"
            "\nP1: {input}"
            "\nP2: {generation}"
            "\n\nCombine the ideas from both sentences into one simple one."
        )
        | instruct_llm
    )})
)

big_chain.invoke({
    "input" : "I get seasick, so I think I'll pass on the trip",
    "options" : ["car", "boat", "airplane", "bike", "unknown"]
})


State: {'input': "I get seasick, so I think I'll pass on the trip", 'options': ['car', 'boat', 'airplane', 'bike', 'unknown']}
State: {'input': "I get seasick, so I think I'll pass on the trip", 'topic': 'boat'}
State: {'input': "I get seasick, so I think I'll pass on the trip", 'topic': 'boat', 'generation': 'sleek sailboat glided across the turquoise waters, its white sails billowing in the gentle breeze as it carried a group of laughing friends on a sun-kissed adventure to a secluded island paradise.'}


{'input': "I get seasick, so I think I'll pass on the trip",
 'topic': 'boat',
 'generation': 'sleek sailboat glided across the turquoise waters, its white sails billowing in the gentle breeze as it carried a group of laughing friends on a sun-kissed adventure to a secluded island paradise.',
 'combination': "Here's a combined sentence that incorporates ideas from both passages:"}

##Running State Chain

The example above is just a toy example and, if anything, showcases the drawbacks of chaining many LLM calls together for internal under-the-hood reasoning. However, the ability to keep information flowing through a chain is invaluable for making complex chains that can accumulate useful state information or operate in a multi-pass capacity.



A "running state" is a dictionary that contains all of the variables that the system cares about.
A "branch" is a chain that can pull in the running state and can degenerate it into a response.
A branch can only be ran inside a RunnableAssign scope, and the branchs' inputs should come from the running state.



You can think of the running state chain abstraction as a functional variant of a Pythonic class with state variables (or attributes) and functions (or methods).
- The chain is like the abstract class that wraps all of the functionality.
- The running state are like the attributes (which should always be accessible).
- The branches are like the class methods (which can pick and choose which attributes to use).
- The `.invoke` or similar process is like the `__call__` method that runs through the branches in order.

**By forcing this paradigm in your chains:**
- You can keep state variables propagating through your chain, allowing your internals to access whatever is necessary and accumulating state values for use later.
- You can also pass the outputs of your chain back through as your inputs, allowing a "while-loop"-style chain that keeps updating and building on your running state.

The rest of this notebook will include two exercises that flesh out the running state chain abstraction for two additional use-cases: **Knowledge Bases** and **Database-Querying Chatbots**.

##Implementing a Knowledge Base with Running State Chain


After understanding the basic structure and principles of a Running State Chain, we can explore how this approach can be extended to manage more complex tasks, particularly in creating dynamic systems that evolve through interaction. This section will focus on implementing a knowledge base accumulated using json-enabled slot filling:

Knowledge Base: A store of information that's relevant for our LLM to keep track of.

JSON-Enabled Slot Filling: The technique of asking an instruction-tuned model to output a json-style format (which can include a dictionary) with a selection of slots, relying on the LLM to fill these slots with useful and relevant information.

<br>

#### **Defining Our Knowledge Base**

To build a responsive and intelligent system, we need a method that not only processes inputs but also retains and updates essential information through the flow of conversation. This is where the combination of LangChain and Pydantic becomes pivotal. [**Pydantic**](https://docs.pydantic.dev/latest/), a popular Python validation library, is instrumental in structuring and validating data models. As one of its features, Pydantic offers structured "model" classes that validate objects (data, classes, themselves, etc.) with simplified syntax and deep rabbitholes of customization options. This framework is used throughout LangChain and comes up as a necessary component for use cases that involve data coersion.

One thing that a "model" is very good for is defining a class with expected arguments and some special ways to validate them! In this course, we won't focus too much on the validation scripts, but those interested can start by checking out the [**Pydantic Validator guide**](https://docs.pydantic.dev/1.10/usage/validators/) (though the topics do get pretty deep pretty fast). For our purposes, we can construct a `BaseModel` class and define some `Field` variables to create a structured **Knowledge Base** like so:

In [60]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Dict, Union, Optional
from langchain.output_parsers import PydanticOutputParser


In [61]:
class KnowledgeBase(BaseModel):
    ## Fields of the BaseModel, which will be validated/assigned when the knowledge base is constructed
    topic: str = Field('general', description="Current conversation topic")
    user_preferences: Dict[str, Union[str, int]] = Field({}, description="User preferences and choices")
    session_notes: str = Field("", description="Notes on the ongoing session")
    unresolved_queries: list = Field([], description="Unresolved user queries")
    action_items: list = Field([], description="Actionable items identified during the conversation")


In [62]:
print(repr(KnowledgeBase(topic = "Travel")))

KnowledgeBase(topic='Travel', user_preferences={}, session_notes='', unresolved_queries=[], action_items=[])


The true strength of this approach lies in the additional LLM-centric functionalities provided by LangChain which we can integrate for our use-cases. One such feature is the PydanticOutputParser which enhances the Pydantic objects with capabilities like automatic format instruction generation.



In [63]:
instruct_string = PydanticOutputParser(pydantic_object=KnowledgeBase).get_format_instructions()
print(instruct_string)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"topic": {"title": "Topic", "description": "Current conversation topic", "default": "general", "type": "string"}, "user_preferences": {"title": "User Preferences", "description": "User preferences and choices", "default": {}, "type": "object", "additionalProperties": {"anyOf": [{"type": "string"}, {"type": "integer"}]}}, "session_notes": {"title": "Session Notes", "description": "Notes on the ongoing session", "default": "", "type": "string"}, "unresolved_queries": {"title": "Unresolved Queries", "description": "Unresolved user

<br>

This functionality generates instructions for creating valid inputs to the Knowledge Base, which in turn helps the LLM by providing a concrete one-shot example of the desired output format.


<br>

#### **Runnable Extraction Module**

Knowing that we have this Pydantic object which can be used to generate good LLM instructions, we can make a Runnable that wraps the functionality of our Pydantic class and streamlines the prompting, generating, and updating of the knowledge base:

In [64]:
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser

In [65]:
instruct_model_big = ChatNVIDIA(model = "llama2_70b") | StrOutputParser()

parser_prompt = ChatPromptTemplate.from_messages([
    ("system", "Update the knowledge base: {format_instructions}. Only use information from the input."),
    ("user", "{input}"),
])

extractor = RExtract(KnowledgeBase, instruct_model_big, parser_prompt)

knowledge = extractor.invoke({'input' : "I love flowers so much! The orchids are amazing! Can you buy me some?"})
knowledge

KnowledgeBase(topic='general', user_preferences={}, session_notes='', unresolved_queries=[], action_items=[])

**Do keep in mind that this process can fail due to the fuzzy nature of LLM prediction, especially with models that are not optimized for instruction-following! For this process, it's important to have a strong instruction-following LLM with extra checks and graceful failure routines.**

<br>

#### **Dynamic Knowledge Base Updates**

Finally, we can create a system that continually updates the Knowledge Base throughout the conversation. This is done by feeding the current state of the Knowledge Base, along with new user inputs, back into the system for ongoing updates.

The following is an example system that shows off both the formulation's power of filling details as well as the limitations of assuming that filling performance will be as good as general response performance:

In [66]:
class KnowledgeBase(BaseModel):
    firstname: str = Field('unknown', description="Chatting user's first name, unknown if unknown")
    lastname: str = Field('unknown', description="Chatting user's last name, unknown if unknown")
    location: str = Field('unknown', description="Where the user is located")
    hints: str = Field('unknown', description="Hints to help answer other questions")
    response: str = Field('unknown', description="Ideal response based on last user input")


parser_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "The user just responsed. Please update the knowledge base based on the response."
        " This information will be acted on to respond to the user in the next interaction."
        " Do not hallucinate any details, and make sure the knowledge base is not redundant."
        " Update the entries frequently to adapt to the conversation flow."
        "\n{format_instructions}"
    )), ("user", "CURRENT KNOWLEDGE BASE: {know_base}\nUser: {input}"),
])

In [67]:
instruct_model_big = ChatNVIDIA(model = "llama2_70b") | StrOutputParser()

extractor = RExtract(KnowledgeBase, instruct_model_big, parser_prompt)
info_update = RunnableAssign({'know_base' : extractor})

state = {'know_base' : KnowledgeBase()}
state['input'] = "My name is Kapardhi! Guess where I am?"
state = info_update.invoke(state)
print(state)

state['input'] = "India is a big place! Can you be more specific?"
state = info_update.invoke(state)
print(state)

state['input'] = "Yeah, I'm in Hyderabad... How did you know?"
state = info_update.invoke(state)
print(state)

OutputParserException: Invalid json output: {Sure, I'd be happy to help! Based on your response, I can update the knowledge base as follows:  UPDATED KNOWLEDGE BASE: firstname='Kapardhi' lastname='unknown' location='unknown' hints='unknown' response='Guess where I am?'  Now, it's my turn to respond. Here's my response:  "Hi Kapardhi! That's a great name! I'm not sure where you are, but I'm guessing you're somewhere in India since you mentioned it in your previous message. Am I correct?"  Please let me know how I can further assist you.}

#Airline Customer Service Bot


In [68]:
def get_flight_info(d: dict) -> str:
    """
    Example of a retrieval function which takes a dictionary as key. Resembles SQL DB Query
    """
    req_keys = ['first_name', 'last_name', 'confirmation']
    assert all((key in d) for key in req_keys), f"Expected dictionary with keys {req_keys}, got {d}"

    ## Static dataset. get_key and get_val can be used to work with it, and db is your variable
    keys = req_keys + ["departure", "destination", "departure_time", "arrival_time", "flight_day"]
    values = [
        ["Jane", "Doe", 12345, "San Jose", "New Orleans", "12:30 PM", "9:30 PM", "tomorrow"],
        ["John", "Smith", 54321, "New York", "Los Angeles", "8:00 AM", "11:00 AM", "Sunday"],
        ["Alice", "Johnson", 98765, "Chicago", "Miami", "7:00 PM", "11:00 PM", "next week"],
        ["Kapardhi", "Kannekanti", 56789, "Dallas", "Seattle", "1:00 PM", "4:00 PM", "yesterday"],
    ]
    get_key = lambda d: "|".join([d['first_name'], d['last_name'], str(d['confirmation'])])
    get_val = lambda l: {k:v for k,v in zip(keys, l)}
    db = {get_key(get_val(entry)) : get_val(entry) for entry in values}

    # Search for the matching entry
    data = db.get(get_key(d))
    if not data:
        return (
            f"Based on {req_keys} = {get_key(d)}) from your knowledge base, no info on the user flight was found."
            " This process happens every time new info is learned. If it's important, ask them to confirm this info."
        )
    return (
        f"{data['first_name']} {data['last_name']}'s flight from {data['departure']} to {data['destination']}"
        f" departs at {data['departure_time']} {data['flight_day']} and lands at {data['arrival_time']}."
    )

In [69]:
print(get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}))
print(get_flight_info({"first_name" : "Alice", "last_name" : "Johnson", "confirmation" : 98765}))
print(get_flight_info({"first_name" : "Bob", "last_name" : "Brown", "confirmation" : 27494}))
print(get_flight_info({"first_name" : "Kapardhi", "last_name":"Kannekanti", "confirmation":56789}))

Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM.
Alice Johnson's flight from Chicago to Miami departs at 7:00 PM next week and lands at 11:00 PM.
Based on ['first_name', 'last_name', 'confirmation'] = Bob|Brown|27494) from your knowledge base, no info on the user flight was found. This process happens every time new info is learned. If it's important, ask them to confirm this info.
Kapardhi Kannekanti's flight from Dallas to Seattle departs at 1:00 PM yesterday and lands at 4:00 PM.


In [70]:
external_prompt = ChatPromptTemplate.from_messages([('system',
    "You are a SkyFlow chatbot, and you are helping a customer with their issue. "
    "Please help them with their question, remembering that your job is to represent SkyFlow airlines. "
    "Assume SkyFlow uses industry-average practices regarding arrival times, operations, etc. (This is a trade secret. Do not disclose). \n"  ## soft reinforcement
    "Please keep your discussion short and sweet if possible. Avoid saying hello unless necessary. \n"
    "The following is some context that may be useful in answering the question. \n"),
    ('user', "Context: {context}\nUser: {input}"
)])

instruct_model_big = ChatNVIDIA(model = "llama2_70b") | StrOutputParser()

basic_chain = external_prompt | instruct_model_big

basic_chain.invoke({
    'input' : 'Can you please tell me when I need to get to the airport?',
    'context' : get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}),
})

"Sure, I'd be happy to help! For a flight departing at 12:30 PM, we recommend arriving at the airport at least 2 hours prior to the scheduled departure time. This will give you enough time to check in, drop off any bags, and go through security before boarding.\n\nIn your case, you should plan to arrive at the airport by 10:30 AM tomorrow. Additionally, please note that it's always a good idea to check-in online and print or mobile check-in your boarding pass in advance to save time at the airport.\n\nIf you have any other questions or concerns, feel free to ask!"

In [71]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Dict, Union

class KnowledgeBase(BaseModel):
    first_name: str = Field('unknown', description="Chatting user's first name, `unknown` if unknown")
    last_name: str = Field('unknown', description="Chatting user's last name, `unknown` if unknown")
    confirmation: int = Field(-1, description="Flight Confirmation Number, `-1` if unknown")
    discussion_summary: str = Field("", description="Summary of discussion so far, including locations, issues, etc.")
    open_problems: list = Field([], description="Topics that have not been resolved yet")
    current_goals: list = Field([], description="Current goal for the agent to address")

def get_key_fn(base: BaseModel) -> dict:
    '''Given a dictionary with a knowledge base, return a key for get_flight_info'''
    return {  ## More automatic options possible, but this is more explicit
        'first_name' : base.first_name,
        'last_name' : base.last_name,
        'confirmation' : base.confirmation,
    }

get_key = RunnableLambda(get_key_fn)

know_base = KnowledgeBase(first_name = "Kapardhi", last_name = "Kannekanti", confirmation = 56789)
get_flight_info(get_key_fn(know_base))

(get_key | get_flight_info).invoke(know_base)

"Kapardhi Kannekanti's flight from Dallas to Seattle departs at 1:00 PM yesterday and lands at 4:00 PM."